# C-SWM 

In [25]:
#Libs
import Base: iterate, length, GC
using HDF5
using Knet
using Statistics: mean,std
using Random
using LinearAlgebra
using Images
using Plots

#Datatype
atype=KnetArray{Float32}

#Includes
include("datasets.jl")
include("cswm.jl")

## Model Definition

In [26]:
#Params
input_ch = 3
hidden_dim = 512
num_objects = 5
embedding_dim = 2
action_dim = 4
sigma = 0.5
hinge = 1.0;

In [27]:
Knet.seed!(42)

MersenneTwister(UInt32[0x0000002a], Random.DSFMT.DSFMT_state(Int32[964434469, 1073036706, 1860149520, 1073503458, 1687169063, 1073083486, -399267803, 1072983952, -909620556, 1072836235  …  -293054293, 1073002412, -1300127419, 1073642642, 1917177374, -666058738, -337596527, 1830741494, 382, 0]), [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], UInt128[0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000  …  0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x0000000000

## Load the pretrained model

In [28]:
model = Knet.load("model_2dshapes_vmnist2.jld2", "model")

ContrastiveSWM(EncoderCNNLarge(Param{KnetArray{Float32,4}}[P(KnetArray{Float32,4}(3,3,3,32)), P(KnetArray{Float32,4}(3,3,32,32)), P(KnetArray{Float32,4}(3,3,32,32)), P(KnetArray{Float32,4}(3,3,32,5))], Param{KnetArray{Float32,4}}[P(KnetArray{Float32,4}(1,1,32,1)), P(KnetArray{Float32,4}(1,1,32,1)), P(KnetArray{Float32,4}(1,1,32,1)), P(KnetArray{Float32,4}(1,1,5,1))], Knet.sigm, NNlib.relu), EncoderMLP(Param{KnetArray{Float32,2}}[P(KnetArray{Float32,2}(512,2500)), P(KnetArray{Float32,2}(512,512)), P(KnetArray{Float32,2}(2,512))], Param{KnetArray{Float32,1}}[P(KnetArray{Float32,1}(512)), P(KnetArray{Float32,1}(512)), P(KnetArray{Float32,1}(2))], NNlib.relu), TransitionGNN(EdgeMLP(Param{KnetArray{Float32,2}}[P(KnetArray{Float32,2}(512,4)), P(KnetArray{Float32,2}(512,512)), P(KnetArray{Float32,2}(512,512))], Param{KnetArray{Float32,1}}[P(KnetArray{Float32,1}(512)), P(KnetArray{Float32,1}(512)), P(KnetArray{Float32,1}(512))], NNlib.relu), NodeMLP(Param{KnetArray{Float32,2}}[P(KnetArray{Floa

In [29]:
#Params
EVAL_DATASET_PATH = "/home/cagan/dev/datasets/moving_mnist/movingmnist_tst.h5"
EVAL_BATCH_SIZE = 100
STEP_SIZE = 5

5

In [30]:
dtst = buildMovingMNISTMultiStepPathDataset(EVAL_DATASET_PATH, EVAL_BATCH_SIZE,5);

In [31]:
o,a,no = first(dtst)

(K32(50,50,3,100)[0.0⋯], [-0.10317132622003555 0.489239901304245 … 0.1398233026266098 0.090704046189785; 0.7571961283683777 -0.5166544318199158 … 0.3028825521469116 -0.13918791711330414; … ; -0.592779278755188 0.3050527274608612 … -0.026741869747638702 -0.16450729966163635; 0.24647584557533264 0.22441205382347107 … 0.1293712556362152 0.9074915051460266]

[0.32774773240089417 -0.3775995671749115 … 0.29655563831329346 -0.5970755219459534; -0.21450795233249664 0.2551420032978058 … -0.17867660522460938 0.46698465943336487; … ; -0.12469091266393661 -0.9081464409828186 … 0.3131658732891083 -0.11418511718511581; -0.6580139994621277 0.6381969451904297 … 0.031022993847727776 0.163753941655159]

[0.708202600479126 0.24687859416007996 … 0.9113793969154358 0.09831277281045914; -0.10907024145126343 0.7420267462730408 … 0.45545992255210876 -0.5958659648895264; … ; -0.3909018635749817 0.14424575865268707 … -0.19250370562076569 0.4614567458629608; 0.2407628297805786 -0.7103050351142883 … -0.2499934583

In [33]:
pred_states = Any[]
next_states = Any[]

num_samples = dtst.dataset_size

for  (k, (obs, action, next_obs)) in enumerate(dtst)
    
    if k % 10 == 0
        
        println("Processed ", k ," batches")
        
    end
    #Obs => (50,50,3,100)
    #Next obs => (50,50,3,100)
    pred_state = model(obs,action[:,:,1]) + model(obs)
    
    for i in 2:STEP_SIZE

        pred_state = pred_state + model.gnn(pred_state,action[:,:,i])
        
    end
    
    pred_state = Array{Float32}(pred_state)
    next_state = Array{Float32}(model(next_obs))
    
    #Pred-state => (2,5,100)
    #Next state => (2,5,100)
    #println(pred_state)
    #println(next_state)
    
    push!(pred_states, pred_state)
    push!(next_states, next_state)
    
end

#Pred state cat => [2,5,10000]
#Next state cat => [2,5,10000]
pred_states = cat(pred_states...,dims=3)
next_states = cat(next_states...,dims=3)
    
#Flatten object/feature dimensions
pred_states = mat(pred_states)  #[10,10000]
next_states = mat(next_states)  #[10,10000]

#Calculate pairwise distances
sizes_1 = (size(pred_states)...,1)
sizes_2 = (sizes_1[1], sizes_1[3], sizes_1[2])

pred_states = reshape(pred_states, sizes_1)
next_states = reshape(next_states, sizes_2)
pred_states = repeat(pred_states, outer=[1,1,1000])
next_states = repeat(next_states, outer=[1,1000,1])

pairwise_distance_matrix = sum((pred_states - next_states).^2, dims=1)[1,:,:]

#Augment pairwise distance matrix
diag_elements = diag(pairwise_distance_matrix)
pairwise_distance_matrix = hcat(diag_elements, pairwise_distance_matrix)


labels = ones(num_samples)
hits_at_1 = 0

indices = []

for i=1:1000
    
    row = pairwise_distance_matrix[i,:]
    ind = sortperm(row)
    
    push!(indices, ind)

end

indices = vcat(indices'...)

Processed 10 batches


1000×1001 Array{Int64,2}:
 799   270   47  349  720  808  451  …  996  195   64  855  732  840   96
 253   521  117  805   59  309  292     808  223  866  571  799  761  392
 101   413  690    1    4  105  570      74  918  299  475  228  765  409
 470   686  370  596  935  450   35     839  845  896  487  553  421  205
 260   436  324  242  911  479  196     840  732  918  255  881  630  765
  17   315  569  417  158  490  150  …  263  195  993  956  365   71  229
 365   848  393  736  277  508  460     724  600   29  763  853   74  417
 503   148   24    5  402  139   86     565  460  795  393  365  589  573
 559   137  582  268  704  612    1     241  230  937  873  697   97  546
 258   349  720  585  505   25  980     131   96  857  123  308  732  855
 651     1   12  231  968  189  232  …  178  863  552  421  647  230  957
 751   643   60  172  985  958  721     914  454  179  875  216  727  718
 287   623  452  490  936  843  158     255  223   71  881  956  697  229
   ⋮        

In [34]:
num_matches = sum(labels .== indices[:,1])
println("Hits @ 1: ", num_matches/num_samples)

Hits @ 1: 0.051


In [35]:
num_matches = sum(labels .== indices[:,1:5])
println("Hits @ 5: ", num_matches/num_samples)

Hits @ 5: 0.185


In [36]:
mxval, mxindx = findmax(indices .== labels,dims=2)
ranks = [ i[2] for i in mxindx ]
reciprocal_ranks = 1 ./ranks
rr_sum = sum(reciprocal_ranks)
println("MRR: ", rr_sum/num_samples)

MRR: 0.13347167186960937
